In [1]:
from __future__ import print_function
import glob, os
import scipy.io as sio
from scipy.io import wavfile
import scipy
import numpy as np
import pandas as pd
from scipy import stats, signal
from numpy.random import seed
from scipy.signal import butter, lfilter, freqz, hilbert
import matplotlib.pyplot as plt
import csv
import librosa
seed(1)

In [2]:
my_path = os.getcwd()
print(my_path)


W:\hno\data\gruppen\Audiologie\at900\Studien\Ivine\My_papers\DNN_Paper\Github_to_Upload\PreProc_folder


In [3]:
fs_if = 16000
fs_eeg_final = 64

win_len_sec = 3  # in sec
hop_len_sec = 1  # 66.66 % overlap
win_len = fs_if*win_len_sec

In [4]:
def List_files(directory,extension):
    os.chdir(directory)
    files=[]
    for file in glob.glob("*.{}".format(extension)):
        files.append(file)

    os.chdir(my_path)
    print('pwd', os.getcwd())
    return files

In [5]:
def store_Audio(datapath):
    
    name_seg = 'audio'  

    audio = wavfile.read(datapath)
    fs_orig = audio[0]
    speech = audio[1]
    
    # added on 25.10
    speech = speech[:, 0:2]/2**15
    
    Att1 = speech[:,0]
    Att2 = speech[:,1]
       
    num_win = np.floor(np.shape(Att1)[0]/(hop_len_sec*fs_if))
    num_win = int(num_win)
    num_win -= 2
    
    print('Get Att1 resampled, shape:',np.shape(Att1),'Get Att2 resampled, shape:',np.shape(Att2), 'num_win', num_win)

    Att_merge_dir = os.path.join(my_path,'Speech_data')
    start_idx = 0
    
    for idx in range(0, num_win):
        stop_idx = start_idx + win_len
#         print(start_idx, stop_idx)

        data_Att1 = Att1[start_idx : stop_idx]
        data_Att2 = Att2[start_idx : stop_idx]
        
        features_att1 = np.abs(librosa.stft(y=data_Att1, n_fft=512, win_length=int(fs_if*32/1000), hop_length= int(fs_if*20/1000), window='hann'))**2
        features_att2 = np.abs(librosa.stft(y=data_Att2, n_fft=512, win_length=int(fs_if*32/1000), hop_length= int(fs_if*20/1000), window='hann'))**2

        if 0:
            plt.plot(data_Att1)
            plt.show()
            plt.imshow(20*np.log10(features_att1), cmap='jet', interpolation='nearest', aspect='auto')
            plt.colorbar()
            plt.show()
            
            plt.plot(data_Att2)
            plt.show()
            plt.imshow(20*np.log10(features_att2), cmap='jet', interpolation='nearest', aspect='auto')
            plt.colorbar()
            plt.show()
        
        features_att1 = np.expand_dims(features_att1, axis=2)
        features_att2 = np.expand_dims(features_att2, axis=2)
        
        features_merged = np.concatenate((features_att1/np.max(abs(features_att1)), features_att2/np.max(abs(features_att2))), axis=2)       
        features_merged = np.transpose(features_merged, (1,0,2))         
        
        np.save(os.path.join(Att_merge_dir, name_seg + str(idx)), features_merged)
        start_idx += int((hop_len_sec*fs_if))
      

In [6]:
audio_path = my_path + '\\'
Name_raw_Audio = List_files(audio_path,'wav')
print(Name_raw_Audio)

for idx in np.arange(0, np.shape(Name_raw_Audio)[0]):
    store_Audio(audio_path + Name_raw_Audio[idx])


pwd W:\hno\data\gruppen\Audiologie\at900\Studien\Ivine\My_papers\DNN_Paper\Github_to_Upload\PreProc_folder
['stim_1min.wav']
Get Att1 resampled, shape: (960000,) Get Att2 resampled, shape: (960000,) num_win 58


In [7]:
def storing_EEG_HI(eeg_path, sub_id, csv_mode):
    
    matstruct_squeezed= sio.loadmat(eeg_path + sub_id)    
    seg_num = 1
    num_win = 58
            
    EEG_dir = os.path.join(my_path,'EEG')
    hop_length = hop_len_sec*fs_eeg_final
    
#      -----   csv_mode   -----   #    
#     'r': open for reading (default)
#     'w': open for writing, truncating the file first
#     'x': open for exclusive creation, failing if the file already exists
#     'a': open for writing, appending to the end of the file if it exists
#     'b': binary mode
#     't': text mode (default)
#     '+': open for updating (reading and writing)    
                
            
    ## Making a EEG Dataset for useful 10 best channels
    valid_idx = [3, 4, 6, 7, 9, 10, 11, 12, 13, 16]
    
    with open(my_path  + '\\Preproc_data.csv', csv_mode) as f:
        writer = csv.writer(f, delimiter=",", lineterminator='\n')

        for seg_id in range(seg_num):
            train_win_ids = np.arange(num_win)
            
            data_struct = matstruct_squeezed['seg'] # Loading EEG data for particular segment
            attention = data_struct['attention'][0][0][0] # Extracting the 'attention' data
            print(attention)
            
            if attention=='male1':
                label=0
            else:
                label=1
                            
            ## Extracting EEG data from the particular segment
            myarray = np.asarray(data_struct['eeg_env']) # convert EEG data to a array
            array = myarray[0][0]
                        
            ## Making a EEG Dataset for useful 10 best channels
            eeg_data_chans = array[:, valid_idx]
            print(np.shape(eeg_data_chans))
            
            tmp_num_win = int(np.shape(eeg_data_chans)[0]/(hop_len_sec*fs_eeg_final)) 
#             print('num_win', tmp_num_win)
            
            for idx in train_win_ids:
                start_idx = int(idx*hop_len_sec*fs_eeg_final)        
                stop_idx = start_idx + int(win_len_sec*fs_eeg_final) 
                                           
                sub_name = sub_id.split('_')[0]
                f_name = sub_name + '_' + 'audio' + str(idx)
#                 print(f_name, start_idx, stop_idx)

                dataset = eeg_data_chans[start_idx:stop_idx, :]
                np.save(os.path.join(EEG_dir,f_name), dataset)
                writer.writerow((f_name, label)) 


In [8]:
EEG_path = my_path + '\\'
Name_EEG_files = List_files(EEG_path, 'mat')
print(Name_EEG_files)

for idx in np.arange(0, np.shape(Name_EEG_files)[0]):
    storing_EEG_HI(EEG_path, Name_EEG_files[idx], 'w')


pwd W:\hno\data\gruppen\Audiologie\at900\Studien\Ivine\My_papers\DNN_Paper\Github_to_Upload\PreProc_folder
['eeg_1min.mat']
male2
(3840, 10)


In [9]:
df_orig = pd.read_csv('Preproc_data.csv')

p = 0.75 
df_train = pd.read_csv('Preproc_data.csv', skiprows=lambda i: i>0 and np.random.random() > p)
df_train.to_csv('train.csv', index=False)

df_temp = pd.concat([df_orig, df_train]).drop_duplicates(keep=False)
num_samples = int(df_temp.shape[0]/2)

df_val = df_temp.head(num_samples)
df_val.to_csv('validation.csv', index=False, header=None)

df_test = df_temp.tail(num_samples)
df_test.to_csv('test.csv', index=False, header=None)